In [1]:
import cvxpy
import numpy as np


In [25]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4
p

array([0.42980446, 0.13434884, 0.83161561, 0.95904662])

In [32]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
prob.solve()

battery.value, action.value

(array([ 5.09510938e-15,  3.00000000e-01,  6.00000000e-01,  4.76849229e-01,
        -3.57914592e-13]),
 array([ 0.3       ,  0.3       , -0.12315077, -0.47684923]))

In [13]:
p

array([0.92376452, 0.32255559, 0.41125936, 0.61318695])

In [ ]:


x_hat = cvxpy.Variable(A.shape[1])
objective = cvxpy.Minimize(cvxpy.norm(x_hat, 1))
    constraints = [A * x_hat == y]
    prob = cvxpy.Problem(objective, constraints)
    prob.solve()
    return np.squeeze(np.array(x_hat.value)), (prob.value, prob.status)